In [1]:
import sys
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import seaborn as sb
import matplotlib.pyplot as plt
import uproot
import time

from itertools import product

#from root_numpy import root2array

from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import QuantileTransformer
import pickle

Welcome to JupyROOT 6.16/00


/Users/jonamotta/anaconda3/envs/python_root/lib/python2.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
Using TensorFlow backend.


In [2]:
########################################
### READ TREES AND CREATE DATAFRAMES ###
########################################


# fix random seed for reproducibility
np.random.seed(1986)

#create DataFrames with the values coming from the trees
file_tau = uproot.open('../bc_jpsi_tau_nu_gen_v2.root')
tree_tau = file_tau['tree;1']
tau  = tree_tau.pandas.df(tree_tau.keys())

file_mu = uproot.open('../bc_jpsi_mu_nu_gen_v2.root')
tree_mu = file_mu['tree;2']
mu  = tree_mu.pandas.df(tree_mu.keys())

n_totMC_mu = mu['run'].count()
n_totMC_tau = tau['run'].count()

n_obs_cms = 7629 # number of Bc events observed by CMS as taken from the Bc(2S) paper where Bc->jpsi+pi
n_norm_data = n_obs_cms / 4.96e-2 # account for the different branching ratio of Bc to pi or to mu
n_signal_data_SM = n_obs_cms * 0.265 * 0.17 / 4.96e-2 # account for the different branching ratio of Bc to pi or to mu
n_signal_data_LFNU = n_obs_cms * 0.71 * 0.17 / 4.96e-2 # account for the different branching ratio of Bc to pi or to mu

In [3]:
############################################
### ADD NECESSARY FEATURES TO DATAFRAMES ###
############################################

n_norm_MC = 0
n_signal_MC = 0

mu_vect = ROOT.TLorentzVector()
mu1_vect = ROOT.TLorentzVector()
mu2_vect = ROOT.TLorentzVector()
mu_mass = 0.10565837 #GeV/c^2
jpsi_mass = 3.096900 #GeV/c^2
c = 2.99e8 #m/s

# in the following, when creating the TLorentzVector of mu, mu1, mu2 we try to account for the reconstruction error
# made by the tracker and the muon system in the value of pT
# we include a gaussian smearing -> we draw a value of pT from a gaussian centered at the value of the MC pT of the
#                                   muon and with sigma 7% of the value of the MC pT (CMS performance)
start = time.time()
i = 0
while i < mu['run'].count():
    # we take only the events for which the muons atre all inside the acceptance of the CMS detector
    if abs(mu.at[i,'mu_eta']) < 2.5 and abs(mu.at[i,'mu1_eta']) < 2.5 and abs(mu.at[i,'mu2_eta']) < 2.5 and mu.at[i,'mu_pt'] > 1.2 and mu.at[i,'mu1_pt'] > 1.2 and mu.at[i,'mu2_pt'] > 1.2:
        # the following ifs look for events in the barrel, else they are in the endcap
        if abs(mu.at[i,'mu_eta']) < 1.2 and mu.at[i,'mu_pt'] > 3:
            mu_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu_pt'], mu.at[i,'mu_pt']*1/100),0),mu.at[i,'mu_eta'],mu.at[i,'mu_phi'],mu_mass)
        else:
            mu_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu_pt'], mu.at[i,'mu_pt']*3/100),0),mu.at[i,'mu_eta'],mu.at[i,'mu_phi'],mu_mass)
        if abs(mu.at[i,'mu1_eta']) < 1.2 and mu.at[i,'mu1_pt'] > 3:
            mu1_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu1_pt'], mu.at[i,'mu1_pt']*1/100),0),mu.at[i,'mu1_eta'],mu.at[i,'mu1_phi'],mu_mass)
        else:
            mu1_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu1_pt'], mu.at[i,'mu1_pt']*3/100),0),mu.at[i,'mu1_eta'],mu.at[i,'mu1_phi'],mu_mass)
        if abs(mu.at[i,'mu2_eta']) < 1.2 and mu.at[i,'mu2_pt'] > 3:
            mu2_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu2_pt'], mu.at[i,'mu2_pt']*1/100),0),mu.at[i,'mu2_eta'],mu.at[i,'mu2_phi'],mu_mass)
        else:
            mu2_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu2_pt'], mu.at[i,'mu2_pt']*3/100),0),mu.at[i,'mu2_eta'],mu.at[i,'mu2_phi'],mu_mass)

        muons = [mu_vect,mu1_vect,mu2_vect]
        muons_charge = [mu.at[i,'mu_charge'],mu.at[i,'mu1_charge'],mu.at[i,'mu2_charge']]
        if muons_charge[0] != muons_charge[1]:
            pair1_vect = muons[0] + muons[1]
            pair2_vect = muons[1] + muons[2]
            flag = True
        else:
            pair1_vect = muons[0] + muons[2]
            pair2_vect = muons[1] + muons[2]
            flag = False
                    
        if (abs(pair1_vect.Mag() - jpsi_mass)) < abs((pair2_vect.Mag() - jpsi_mass)):
            if pair1_vect.Mag() < 3.3 and pair1_vect.Mag() > 2.9:
                if flag == True and muons[0].Pt() > 4 and muons[1].Pt() > 4:
                    n_norm_MC += 1
                if flag == False and muons[0].Pt() > 4 and muons[2].Pt() > 4:
                    n_norm_MC += 1     
        else:
            if pair1_vect.Mag() < 3.3 and pair1_vect.Mag() > 2.9:
                if flag == True and muons[0].Pt() > 4 and muons[2].Pt() > 4:
                    n_norm_MC += 1
                if flag == False and muons[1].Pt() > 4 and muons[2].Pt() > 4:
                    n_norm_MC += 1 
                    
                
    i +=1

# ------ end one loop, start the other ------ # 
    
i = 0
while i < tau['run'].count():
    # we take only the events for which the muons atre all inside the acceptance of the CMS detector
    if abs(tau.at[i,'mu_eta']) < 2.5 and abs(tau.at[i,'mu1_eta']) < 2.5 and abs(tau.at[i,'mu2_eta']) < 2.5 and tau.at[i,'mu_pt'] > 1.2 and tau.at[i,'mu1_pt'] > 1.2 and tau.at[i,'mu2_pt'] > 1.2:
        # the following ifs look for events in the barrel, else they are in the endcap
        if abs(tau.at[i,'mu_eta']) < 1.2 and tau.at[i,'mu_pt'] > 3:
            mu_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu_pt'], tau.at[i,'mu_pt']*1/100),0),tau.at[i,'mu_eta'],tau.at[i,'mu_phi'],mu_mass)
        else:
            mu_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu_pt'], tau.at[i,'mu_pt']*3/100),0),tau.at[i,'mu_eta'],tau.at[i,'mu_phi'],mu_mass)
        if abs(tau.at[i,'mu1_eta']) < 1.2 and tau.at[i,'mu1_pt'] > 3:
            mu1_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu1_pt'], tau.at[i,'mu1_pt']*1/100),0),tau.at[i,'mu1_eta'],tau.at[i,'mu1_phi'],mu_mass)
        else:
            mu1_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu1_pt'], tau.at[i,'mu1_pt']*3/100),0),tau.at[i,'mu1_eta'],tau.at[i,'mu1_phi'],mu_mass)
        if abs(tau.at[i,'mu2_eta']) < 1.2 and tau.at[i,'mu2_pt'] > 3:
            mu2_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu2_pt'], tau.at[i,'mu2_pt']*1/100),0),tau.at[i,'mu2_eta'],tau.at[i,'mu2_phi'],mu_mass)
        else:
            mu2_vect.SetPtEtaPhiM(max(np.random.normal(tau.at[i,'mu2_pt'], tau.at[i,'mu2_pt']*3/100),0),tau.at[i,'mu2_eta'],tau.at[i,'mu2_phi'],mu_mass)

        muons = [mu_vect,mu1_vect,mu2_vect]
        muons_charge = [tau.at[i,'mu_charge'],tau.at[i,'mu1_charge'],tau.at[i,'mu2_charge']]
        if muons_charge[0] != muons_charge[1]:
            pair1_vect = muons[0] + muons[1]
            pair2_vect = muons[1] + muons[2]
            flag = True
        else:
            pair1_vect = muons[0] + muons[2]
            pair2_vect = muons[1] + muons[2]
            flag = False         
                    
        if (abs(pair1_vect.Mag() - jpsi_mass)) < abs((pair2_vect.Mag() - jpsi_mass)):
            if pair1_vect.Mag() < 3.3 and pair1_vect.Mag() > 2.9:
                if flag == True and muons[0].Pt() > 4 and muons[1].Pt() > 4:
                    n_signal_MC += 1
                if flag == False and muons[0].Pt() > 4 and muons[2].Pt() > 4:
                    n_signal_MC += 1     
        else:
            if pair1_vect.Mag() < 3.3 and pair1_vect.Mag() > 2.9:
                if flag == True and muons[0].Pt() > 4 and muons[2].Pt() > 4:
                    n_signal_MC += 1
                if flag == False and muons[1].Pt() > 4 and muons[2].Pt() > 4:
                    n_signal_MC += 1  
                
                
    i +=1
    
end = time.time()
print 'Running time to add the new vars to the df = %.1f'%(end - start)

Running time to add the new vars to the df = 989.1


In [4]:
#################################
### CALCULATE MU SCALE FACTOR ###
#################################

# we have to account for the muon ID efficiency
mu_scale_factor = n_norm_data / (n_norm_MC*0.96**3)

print 'MU SCALE FACTOR = ', mu_scale_factor

MU SCALE FACTOR =  49.9566140696


In [6]:
##################################
### CALCULATE TAU SCALE FACTOR ###
##################################


# we have to account for the different luminosity at which the MC simulations were produced
MClumi_scale_factor_SM = n_totMC_mu * 0.265 * 0.17 / n_totMC_tau
MClumi_scale_factor_LFNU = n_totMC_mu * 0.71 * 0.17 / n_totMC_tau
tau_scale_factor_SM = n_signal_data_SM / (n_signal_MC*MClumi_scale_factor_SM*0.96**3)
tau_scale_factor_LFNU = n_signal_data_LFNU / (n_signal_MC*MClumi_scale_factor_LFNU*0.96**3)

print 'TAU SCALE FACTOR IN SM PREDICTION = ', tau_scale_factor_SM
print 'TAU SCALE FACTOR IN LFNU PREDICTION = ', tau_scale_factor_LFNU

TAU SCALE FACTOR IN SM PREDICTION=  235.92544374041478
TAU SCALE FACTOR IN LFNU PREDICTION=  235.9254437404148


In [9]:
print mu_scale_factor * MClumi_scale_factor_LFNU / 0.71

30.82785712448828
